In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sub = pd.read_parquet("/kaggle/input/turkiye-is-bankasi-ml-challenge-5/submission_sample_final.parquet")
train = pd.read_parquet("/kaggle/input/turkiye-is-bankasi-ml-challenge-5/train_final.parquet")

In [ ]:
train.head(5)

# 1- Convert Target 

I would like to work with target at First.
I have tow choises 
1- convert it to one feature for example 000100110
2- 3 features 000001,00010,0001000

In [ ]:
train["targ_arr_1"] = None
train["targ_arr_3"] = None

In [ ]:
# get unique values
menues = [menus.split(",")  for menus in train.target.unique()]

uniq_menus = []
for m in menues:
    for menu in m:
        num = menu.strip()
        if num not in uniq_menus:

            uniq_menus.append(num)
uniq_menus.sort()
print(uniq_menus)


In [ ]:
# create index maping
mapping = {}
for i,menu in enumerate(uniq_menus):
    mapping[menu] = i
    
print(mapping)

In [ ]:
# write a function to convert menus
def convert_target(target:str,mapping:dict, out_type="1"):
    """ To Get 3 arrays change "1" to "3"
    """
    target_li = target.split(",")
    
    # output arr
    out = np.zeros(9)
    
    out_3 = []
    for menu in target_li:
        idx = mapping[menu.strip()]
        out[idx] = 1
        if out_type == "3":
            out_3.append(out) #append to the tupple
            out = np.zeros(9)
            
    
    if out_type == "3":
        return np.array(out_3).transpose()
    
    
    return out
        

In [ ]:
# test the function
import random
rand = random.randint(0,len(train))

value = train.target[rand]
print(value)

print("Get Target as 1 array :", convert_target(value,mapping))
print("Get Target as 3 array :", convert_target(value,mapping,out_type="3" ))
print("Get Target as 3 array :", convert_target(value,mapping,out_type="3" ).shape)


In [ ]:
train["targ_arr_1"] = train.target.apply(lambda x: convert_target(x,mapping))
train[["target","targ_arr_1"]].head(5)

In [ ]:
train["targ_arr_3"] = train.target.apply(lambda x: convert_target(x,mapping,"3"))
train[["target","targ_arr_3"]].head(5)
# the arrays are 3 columns

# 2- Feature Selection and Visualiziation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.dtypes

In [ ]:
df = train.copy()
df.drop( columns=["target","targ_arr_1","targ_arr_3"], axis = 1, inplace= True)
df.columns

In [ ]:
numerical_features = df.copy()
numerical_features.drop(["month",'carrier',
       'devicebrand',], axis=1,inplace = True)


In [ ]:
categorical_columns = ["month",'carrier',
       'devicebrand',]

# Selecting the top 10 most occurring categories from 'carrier' and 'devicebrand' columns
top_n = 10

fig, axs = plt.subplots(1, 3, figsize=(18, 9))

for i, column in enumerate(categorical_columns):
    value_counts = train[column].value_counts().nlargest(top_n)
    axs[i].pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%', startangle=140)
    axs[i].set_title(f'Top {top_n} {column} kategorisi')

plt.tight_layout()
plt.show()

In [ ]:
for col in categorical_columns:
    
  counts = train[col].value_counts()
  counts_bigger_001 = counts.loc[counts/len(train) > 0.005]
  print(" counts_bigger_0.001 : ", counts_bigger_001)
  print(" count of them  : ", len(counts_bigger_001))
